In [ ]:
import os
import sys

os.chdir('..')

parent_directory = os.getcwd()
sys.path.append(parent_directory)

In [ ]:
from src.rea_beta_backend_toolsets.services.user_agents.user_agent import UserAgentRotationManager
from src.rea_beta_backend_toolsets.services.proxies.proxy import ProxyRotationManager
from typing import Optional
import requests

class RequestManager:
    _user_agent_rotation_manager_cache = None
    _proxy_rotation_manager_cache = None

    def __init__(self, user_agents: Optional[list[str]] = [], proxies: list[str] = []):
        if RequestManager._user_agent_rotation_manager_cache is None:
            RequestManager._user_agent_rotation_manager_cache = UserAgentRotationManager(user_agents)
        self.user_agent_rotation_manager = RequestManager._user_agent_rotation_manager_cache

        if RequestManager._proxy_rotation_manager_cache is None:
            RequestManager._proxy_rotation_manager_cache = ProxyRotationManager(proxies)
        self.proxy_rotation_manager = RequestManager._proxy_rotation_manager_cache

     
        for i in self.proxy_rotation_manager.valid_proxies:
            print(i)
        print("=========")
        

    def scrape(self, url: str, headers: Optional[dict] = None, params: Optional[dict] = None) -> Optional[requests.Response]:
        proxy = self.proxy_rotation_manager.get_next_proxy().proxy
        user_agent = self.user_agent_rotation_manager.get_next_user_agent()
        if headers is None:
            headers = {}
        headers.update({"User-Agent": user_agent})
        proxies = {
            'http': f'http://{proxy}',
            'https': f'http://{proxy}'
        } 
        print(proxies)
        response = requests.get(url, headers=headers, proxies=proxies, params=params, timeout=10)
        response.raise_for_status()
        return response

request_manager = RequestManager(proxies=[])
response = request_manager.scrape("https://example.com")
if response:
    print(response.text)
user_agent_rotation_manager = UserAgentRotationManager()

In [ ]:
response = request_manager.scrape("https://example.com")
if response:
    print(response.text)
user_agent_rotation_manager = UserAgentRotationManager()